<a href="https://colab.research.google.com/github/HueyVault/study_NLPs/blob/main/codes/DeepLearnings/11_04_generativeLLM_HF_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# 데이터 및 fine tunning 된 모델 셋팅
# !ls -al
# !unzip ./chapter_6_withvLLMme_finetuned_model.zip
# !unzip ./chapter_6_withvLLMme_preprocess.zip
# !ls -al ../input/text2sql/data
# !cp -rf ../input/text2sql/data ./
!ls -al ./data

total 50468
drwxr-xr-x 2 root root     4096 Feb 11 02:13 .
drwxr-xr-x 4 root root     4096 Feb 11 02:13 ..
-rw-r--r-- 1 root root 51668571 Feb 11 02:13 train.csv


In [3]:
import os
# 환경 변수에 따라 파일 경로 설정
if os.getenv('KAGGLE_KERNEL_RUN_TYPE') is not None:
    # Kaggle 환경
    print("kaggle")
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    os.environ['HF_TOKEN']= user_secrets.get_secret("HF_TOKEN")
    os.environ['OPENAI_API_KEY'] = user_secrets.get_secret("OPENAI_API_KEY")

    # file path
    train_file_path =  f'/kaggle/input/house-prices-advanced-regression-techniques/train.csv'
    test_file_path = f'/kaggle/input/house-prices-advanced-regression-techniques/test.csv'

elif 'google.colab' in str(get_ipython()):
    print("colab")
    from google.colab import userdata
    os.environ['HF_TOKEN'] = userdata.get("HF_TOKEN")
    os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")

else:
    # Docker 환경
    print("local")
    train_file_path = "../../datasets/train.csv"
    test_file_path = "../../datasets/test.csv"


kaggle


## 자연어 평가
- EM (Extract Match) : 텍스트 매칭(정규식, 의미, 통계 매칭)
- EX (Execution Accuracy) : 개발 환경에서 실행
- GPT-4 활용


In [4]:
!pip install transformers==4.40.1 bitsandbytes==0.43.1 accelerate==0.29.3 datasets==2.19.0 tiktoken==0.6.0 -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 62.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 85.8 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.0 which is incompatible.
gcsfs 2024.10.0 requires f

In [14]:
import pandas as pd
train_file_path='./data/train.csv'
df_text2sql = pd.read_csv(train_file_path)
df_text2sql.head()

,db_id,context,question,answer,text
0,3,"CREATE TABLE categories (\n ""category_id"" S...",카테고리 이름을 길이가 긴 순서로 정렬하되 가장 긴 이름 3개만 보여줘,SELECT name FROM categories ORDER BY LENGTH(na...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
1,4,"CREATE TABLE users (\n ""user_id"" SERIAL PRI...",카드로 결제한 사용자 중에 이름이 '김'씨씨인 사용자의 이메일과 카드 번호를 찾아주세요.,"SELECT u.email, pm.card_number FROM users AS u...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2,4,"CREATE TABLE subscriptions (\n ""subscriptio...",시작일이 2022년 1월 1일 이후인 유효한 구독의 유저 아이디를 보여줘,SELECT DISTINCT user_id FROM subscriptions WHE...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
3,3,"CREATE TABLE coupons (\n ""coupon_id"" SERIAL...",각기 다른 할인 금액의 쿠폰이 몇 개 있는지 그 수를 나타내줘,"SELECT discount_amount, COUNT(coupon_id) FROM ...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
4,4,"CREATE TABLE users (\n ""user_id"" SERIAL PRI...",리뷰 데이터 중 평점(rating)이 5점인 리뷰에 해당하는 모든 사용자 데이터 조...,SELECT * FROM reviews CROSS JOIN users WHERE r...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...


In [16]:
df_text2sql.loc[1,'text']

'당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.\n\n### DDL:\nCREATE TABLE users (\n    "user_id" SERIAL PRIMARY KEY,\n    "email" VARCHAR(255) UNIQUE NOT NULL,\n    "password_hash" VARCHAR(255) NOT NULL,\n    "name" VARCHAR(255),\n    "created_at" TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,\n    "updated_at" TIMESTAMP\n);\nCREATE TABLE payment_methods (\n    "payment_method_id" SERIAL PRIMARY KEY,\n    "user_id" INTEGER REFERENCES users(user_id),\n    "card_number" VARCHAR(19),\n    "expiry_date" DATE,\n    "billing_address" TEXT,\n    "payment_type" VARCHAR(50)\n);\n\n### Question:\n카드로 결제한 사용자 중에 이름이 \'김\'씨씨인 사용자의 이메일과 카드 번호를 찾아주세요.\n\n### SQL:\nSELECT u.email, pm.card_number FROM users AS u JOIN payment_methods AS pm ON u.user_id = pm.user_id WHERE u.name LIKE \'김%\' AND pm.payment_type = \'카드\';'

## fine tuning 된 모델 이용한 응답 만들기

In [18]:

import torch
from transformers import BitsAndBytesConfig, pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
def make_inference_pipeline(model_id):
    base_model = 'beomi/Yi-Ko-6B'
    # tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    # 버전 호환 문제로 삭제
    # # 양자화(모델 사이즈 축소)
    # quantization_config = BitsAndBytesConfig(
    #     load_in_4bit=True,
    #     bnb_4bit_compute_dtype=torch.float16
    # )
    # model
    model = AutoModelForCausalLM.from_pretrained(model_id,
                                                 # quantization_config=quantization_config,
                                                 load_in_4bit=True,
                                                 bnb_4bit_compute_dtype=torch.float16, # 양자화 정의
                                                 device_map='auto'
                                                )

    
    # pipeline : 예측 초기화 설정
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

    return pipe

In [19]:
finetuning_model_id = 'HueyVault/yi-ko-6b-text2sql'
hf_pipe = make_inference_pipeline(finetuning_model_id)

tokenizer_config.json:   0%|          | 0.00/9.51k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [20]:
def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.
### DDL:
{ddl}
### Question:
{question}
### SQL:
{query}"""
    return prompt

In [21]:
# 필요한 컬럼 추출해 프롬프트 명령어 만들기
from tqdm import tqdm

for idx, row in tqdm(df_text2sql.iterrows(),
                     total = len(df_text2sql),
                     desc='Generating prompt'): # DataFrame row return
    prompt_command = make_prompt(row['context'],
                         row['question']
                        #  row['answer']
                        )
    df_text2sql.loc[idx, 'prompt'] = prompt_command
    pass
df_text2sql.head()

Generating prompt: 100%|██████████| 33876/33876 [00:07<00:00, 4606.41it/s]


,db_id,context,question,answer,text,prompt
0,3,"CREATE TABLE categories (\n ""category_id"" S...",카테고리 이름을 길이가 긴 순서로 정렬하되 가장 긴 이름 3개만 보여줘,SELECT name FROM categories ORDER BY LENGTH(na...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
1,4,"CREATE TABLE users (\n ""user_id"" SERIAL PRI...",카드로 결제한 사용자 중에 이름이 '김'씨씨인 사용자의 이메일과 카드 번호를 찾아주세요.,"SELECT u.email, pm.card_number FROM users AS u...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2,4,"CREATE TABLE subscriptions (\n ""subscriptio...",시작일이 2022년 1월 1일 이후인 유효한 구독의 유저 아이디를 보여줘,SELECT DISTINCT user_id FROM subscriptions WHE...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
3,3,"CREATE TABLE coupons (\n ""coupon_id"" SERIAL...",각기 다른 할인 금액의 쿠폰이 몇 개 있는지 그 수를 나타내줘,"SELECT discount_amount, COUNT(coupon_id) FROM ...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
4,4,"CREATE TABLE users (\n ""user_id"" SERIAL PRI...",리뷰 데이터 중 평점(rating)이 5점인 리뷰에 해당하는 모든 사용자 데이터 조...,SELECT * FROM reviews CROSS JOIN users WHERE r...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...


### 파인튜닝된 모델에 prompt로 질문과 답 얻기

In [22]:
example = df_text2sql.loc[1,'prompt']

In [24]:
results = hf_pipe(example, do_sample=False,
       return_full_text=False, max_length=512, truncation=True)
results

[{'generated_text': "SELECT u.email, p.card_number FROM users AS u JOIN payment_methods AS p ON u.user_id = p.user_id WHERE u.name LIKE '김%';"}]

In [25]:
results[0]['generated_text']

"SELECT u.email, p.card_number FROM users AS u JOIN payment_methods AS p ON u.user_id = p.user_id WHERE u.name LIKE '김%';"

In [28]:
prompts = df_text2sql['prompt'].tolist()
result_gen_sqls = []

for prompt in tqdm(prompts, desc="Generating SQL"):
    result = hf_pipe(prompt,
                    do_sample=False,
                    return_full_text=False,
                    max_new_tokens=512,
                    truncation=True)
    print(result)
    result_gen_sqls.append(result[0]['generated_text'])
df_text2sql['gen_sql'] = result_gen_sqls

Generating SQL:   0%|          | 1/33876 [00:01<17:06:56,  1.82s/it]

[{'generated_text': 'SELECT * FROM categories ORDER BY LENGTH(name) DESC LIMIT 3;'}]


Generating SQL:   0%|          | 2/33876 [00:05<25:25:43,  2.70s/it]

[{'generated_text': "SELECT u.email, p.card_number FROM users AS u JOIN payment_methods AS p ON u.user_id = p.user_id WHERE u.name LIKE '김%';"}]


Generating SQL:   0%|          | 3/33876 [00:07<21:58:04,  2.33s/it]

...

[{'generated_text': 'SELECT * FROM reviews r1 JOIN reviews r2 ON r1.user_id = r2.user_id WHERE r1.user_id <> r2.user_id;'}]


Generating SQL:   0%|          | 22/33876 [00:46<17:17:52,  1.84s/it]

[{'generated_text': 'SELECT status FROM orders WHERE delivery_address IS NULL;'}]


Generating SQL:   0%|          | 22/33876 [00:48<20:52:10,  2.22s/it]


KeyboardInterrupt: 

In [45]:
df_text2sql_50 = df_text2sql[:50]
prompts = df_text2sql_50['prompt'].tolist()
len(prompts)

50

In [46]:
prompts = df_text2sql_50['prompt'].tolist()
result_gen_sqls = []

for prompt in tqdm(prompts, desc="Generating SQL"):
    result = hf_pipe(prompt,
                    do_sample=False,
                    return_full_text=False,
                    max_new_tokens=512,
                    truncation=True)
    
    result_gen_sqls.append(result[0]['generated_text'])

Generating SQL: 100%|██████████| 50/50 [02:05<00:00,  2.51s/it]


ValueError: Length of values (50) does not match length of index (33876)

In [47]:
df_text2sql_50['gen_sql'] = result_gen_sqls

<ipython-input-47-b724ef8be73f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text2sql_50['gen_sql'] = result_gen_sqls


In [48]:
df_text2sql_50.head()

,db_id,context,question,answer,text,prompt,gen_sql
0,3,"CREATE TABLE categories (\n ""category_id"" S...",카테고리 이름을 길이가 긴 순서로 정렬하되 가장 긴 이름 3개만 보여줘,SELECT name FROM categories ORDER BY LENGTH(na...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...,SELECT * FROM categories ORDER BY LENGTH(name)...
1,4,"CREATE TABLE users (\n ""user_id"" SERIAL PRI...",카드로 결제한 사용자 중에 이름이 '김'씨씨인 사용자의 이메일과 카드 번호를 찾아주세요.,"SELECT u.email, pm.card_number FROM users AS u...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...,"SELECT u.email, p.card_number FROM users AS u ..."
2,4,"CREATE TABLE subscriptions (\n ""subscriptio...",시작일이 2022년 1월 1일 이후인 유효한 구독의 유저 아이디를 보여줘,SELECT DISTINCT user_id FROM subscriptions WHE...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...,SELECT user_id FROM subscriptions WHERE start_...
3,3,"CREATE TABLE coupons (\n ""coupon_id"" SERIAL...",각기 다른 할인 금액의 쿠폰이 몇 개 있는지 그 수를 나타내줘,"SELECT discount_amount, COUNT(coupon_id) FROM ...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...,SELECT COUNT(*) FROM coupons GROUP BY discount...
4,4,"CREATE TABLE users (\n ""user_id"" SERIAL PRI...",리뷰 데이터 중 평점(rating)이 5점인 리뷰에 해당하는 모든 사용자 데이터 조...,SELECT * FROM reviews CROSS JOIN users WHERE r...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...,SELECT * FROM users CROSS JOIN reviews WHERE r...


In [27]:
result_gen_sqls[0][0]['generated_text']

'SELECT name FROM categories ORDER BY LENGTH(name) DESC LIMIT 3;'

### 평가 위한 프롬프트 작성

In [34]:
import json
import pandas as pd
from pathlib import Path
# gpt_model = "gpt-4-turbo-preview"
gpt_model = "gpt-4o-mini"
def make_requests_for_gpt_evaluation(df, filename, dir='requests'):
  if not Path(dir).exists():
      Path(dir).mkdir(parents=True)
  prompts = []
  for idx, row in df.iterrows():
      prompts.append("""Based on below DDL and Question, evaluate gen_sql can resolve Question. If gen_sql and gt_sql do equal job, return "yes" else return "no". Output JSON Format: {"resolve_yn": ""}""" + f"""
          DDL: {row['context']}
          Question: {row['question']}
          gt_sql: {row['answer']}
          gen_sql: {row['gen_sql']}"""
          )
  jobs = [{"model": gpt_model, "response_format" : { "type": "json_object" }, "messages": [{"role": "system", "content": prompt}]} for prompt in prompts]
  with open(Path(dir, filename), "w") as f:
      for job in jobs:
          json_string = json.dumps(job)
          f.write(json_string + "\n")



In [35]:
eval_filename = 'text2sql_evaluation.jsonl'

In [52]:
# 프롬프트 일괄 작성 위한 function 호출과 파일 작성(parallel하게 openai api 호출 위해 )
make_requests_for_gpt_evaluation(df_text2sql_50, eval_filename)

In [53]:
!wget https://raw.githubusercontent.com/openai/openai-cookbook/refs/heads/main/examples/api_request_parallel_processor.py

--2025-02-11 03:08:00--  https://raw.githubusercontent.com/openai/openai-cookbook/refs/heads/main/examples/api_request_parallel_processor.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21891 (21K) [text/plain]
Saving to: ‘api_request_parallel_processor.py’

api_request_paralle 100%[===================>]  21.38K  --.-KB/s    in 0.003s  

2025-02-11 03:08:00 (7.02 MB/s) - ‘api_request_parallel_processor.py’ saved [21891/21891]



In [54]:
!mkdir results

In [56]:
!ls -al ./requests

total 68
drwxr-xr-x 2 root root  4096 Feb 11 03:07 .
drwxr-xr-x 6 root root  4096 Feb 11 03:08 ..
-rw-r--r-- 1 root root 59162 Feb 11 03:07 text2sql_evaluation.jsonl


In [57]:
# GPT-4 평가 수행
eval_filepath = 'text2sql_evaluation.jsonl'
# cl100k_base 토크나이즈 버전

!python api_request_parallel_processor.py \
--requests_filepath requests/{eval_filepath}  \
--save_filepath results/{eval_filepath} \
--request_url https://api.openai.com/v1/chat/completions \
--max_requests_per_minute 2500 \
--max_tokens_per_minute 100000 \
--token_encoding_name cl100k_base \
--max_attempts 5 \
--logging_level 20

### openAI 결과 (results/text2sql_evaluation.jsonl)

In [58]:
def change_jsonl_to_csv(input_file, output_file, prompt_column="prompt", response_column="response"):
    prompts = []
    responses = []
    with open(input_file, 'r') as json_file:
        for data in json_file:
            prompts.append(json.loads(data)[0]['messages'][0]['content'])
            responses.append(json.loads(data)[1]['choices'][0]['message']['content'])
    df = pd.DataFrame({prompt_column: prompts, response_column: responses})
    df.to_csv(output_file, index=False)
    return df

In [59]:
evaluation_df = change_jsonl_to_csv(input_file=f'results/{eval_filepath}',
                    output_file=f'results/yi_ko_6b_eval.csv',
                    prompt_column='prompt',
                    response_column='resolve_yn')

In [60]:
evaluation_read_df = pd.read_csv(f'results/yi_ko_6b_eval.csv')
evaluation_read_df.head()

,prompt,resolve_yn
0,"Based on below DDL and Question, evaluate gen_...","{""resolve_yn"": ""no""}"
1,"Based on below DDL and Question, evaluate gen_...","{""resolve_yn"": ""no""}"
2,"Based on below DDL and Question, evaluate gen_...","{""resolve_yn"": ""no""}"
3,"Based on below DDL and Question, evaluate gen_...","{""resolve_yn"": ""no""}"
4,"Based on below DDL and Question, evaluate gen_...","{""resolve_yn"": ""yes""}"


In [61]:
evaluation_df['resolve_yn'] = evaluation_df['resolve_yn'].apply(lambda x : json.loads(x)['resolve_yn'])

In [62]:
correct_sql_number = evaluation_df.query("resolve_yn=='yes'").shape[0]
correct_sql_number

20

In [63]:
df_text2sql_50[["answer","gen_sql"]]

,answer,gen_sql
0,SELECT name FROM categories ORDER BY LENGTH(na...,SELECT * FROM categories ORDER BY LENGTH(name)...
1,"SELECT u.email, pm.card_number FROM users AS u...","SELECT u.email, p.card_number FROM users AS u ..."
2,SELECT DISTINCT user_id FROM subscriptions WHE...,SELECT user_id FROM subscriptions WHERE start_...
3,"SELECT discount_amount, COUNT(coupon_id) FROM ...",SELECT COUNT(*) FROM coupons GROUP BY discount...
4,SELECT * FROM reviews CROSS JOIN users WHERE r...,SELECT * FROM users CROSS JOIN reviews WHERE r...
5,SELECT * FROM recommendations WHERE content_id...,SELECT * FROM recommendations WHERE content_id...
6,SELECT * FROM attachments WHERE uploaded_at >=...,SELECT * FROM attachments WHERE uploaded_at >=...
7,SELECT CAST(cart_id AS TEXT) FROM cart_items W...,SELECT CAST(cart_id AS TEXT) FROM cart_items W...
8,"SELECT restaurant_id, SUM(total_price) AS tota...","SELECT restaurant_id, SUM(total_price) AS tota..."
9,SELECT * FROM order_details GROUP BY book_id H...,SELECT * FROM order_details WHERE quantity >= 5;


In [ ]:
'''
결과 확인 후 성능 향상 방안
1. 양질의 데이터셋 사용(지금 데이터는 오류도 포함되어 있다)
2. 더 성능 좋은 모델 사용
3. 

'''